ДЗ - обучить при помощи tf (keras) API любое ДЗ по обучениею классических моделей. 
Можно сдавать на торче.
Дедлайн 09.03

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
import numpy as np
import matplotlib.pyplot as plt
#from scipy.misc import toimage
%matplotlib inline 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [4]:
!pip install tensorflow[and-cuda]
#!pip install numpy

INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    tensorflow[and-cuda] 2.18.0 depends on nvidia-nccl-cu12==2.21.5; extra == "and-cuda"
    tensorflow[and-cuda] 2.17.1 depends on nvidia-nccl-cu12==2.19.3; extra == "and-cuda"
    tensorflow[and-cuda] 2.17.0 depends on nvidia-nccl-cu12==2.19.3; extra == "and-cuda"
    tensorflow[and-cuda] 2.16.2 depends on nvidia-nccl-cu12==2.19.3; extra == "and-cuda"
    tensorflow[and-cuda] 2.16.1 depends on nvidia-nccl-cu12==2.19.3; extra == "and-cuda"

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict



ERROR: Cannot install tensorflow[and-cuda]==2.16.1, tensorflow[and-cuda]==2.16.2, tensorflow[and-cuda]==2.17.0, tensorflow[and-cuda]==2.17.1 and tensorflow[and-cuda]==2.18.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


Загрузим датасет титаника.
Удалим данные с пропусками.

In [ ]:
np.set_printoptions(edgeitems=30, linewidth=110000, 
    formatter=dict(float=lambda x: "%.3g" % x))        
# Создаем усеченный датафрейм для дальнейшей обработки. Уберем данные о номере билета, номере каюты (мало данных), :
df_train = pd.read_csv("titanic_train.csv",encoding='Windows-1251' ,on_bad_lines='skip',
                    #index_col='client_id' ,
                    usecols=[1,2,4,5, 6,7,9,11]
                    )
print("записей до удаления пропусков",len(df_train))
df_train = df_train.dropna()
print("записей до удаления пропусков",len(df_train))




записей до удаления пропусков 891
записей до удаления пропусков 712


Преобразуем категориальные переменные в цифры:

In [23]:
from sklearn.utils import resample
for col in ['Sex', 'Embarked']:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
print(df_train)

minority_class = df_train[df_train.Survived == 1]
majority_class = df_train[df_train.Survived == 0]
#print(len(minority_class))
#print(len(majority_class))
minority_upsampled = resample(minority_class, 
                              replace=True,  # Sample with replacement
                              n_samples=len(majority_class),  # Equalize class sizes
                              random_state=42)
# Combine the majority class with the upsampled minority class
df_train = pd.concat([majority_class, minority_upsampled])
print(len(df_train))

     Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0           0       3    1  22.0      1      0   7.2500         2
1           1       1    0  38.0      1      0  71.2833         0
2           1       3    0  26.0      0      0   7.9250         2
3           1       1    0  35.0      1      0  53.1000         2
4           0       3    1  35.0      0      0   8.0500         2
..        ...     ...  ...   ...    ...    ...      ...       ...
885         0       3    0  39.0      0      5  29.1250         1
886         0       2    1  27.0      0      0  13.0000         2
887         1       1    0  19.0      0      0  30.0000         2
889         1       1    1  26.0      0      0  30.0000         0
890         0       3    1  32.0      0      0   7.7500         1

[712 rows x 8 columns]
848


In [42]:
y = df_train['Survived']
X = df_train[[ 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)
tf.__version__
tf.config.list_physical_devices('GPU')

[]

In [49]:
#print(model.summary())
adam = tf.keras.optimizers.Adam(learning_rate=0.03)
model = Sequential()
model.add(Dense(7, input_dim=X.shape[1], activation="relu"))
model.add(Dense(3, activation="relu"))
#model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy","AUC"])


model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy','auc'])
history = model.fit(X_train, y_train, batch_size=45, epochs=20, verbose = 1)

score = model.evaluate(X_test, y_test, verbose=1)
print("\nTest loss :", score[0])
print('Test   accuracy:', score[1])
print('Test   auc:', score[2])
model.evaluate

Epoch 1/20


d:\Soft\ProgramFiles\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5431 - auc: 0.5987 - loss: 1.2543   
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5527 - auc: 0.6941 - loss: 0.6654 
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6738 - auc: 0.6926 - loss: 0.6568 
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6494 - auc: 0.7180 - loss: 0.6420 
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7044 - auc: 0.7485 - loss: 0.6074 
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6902 - auc: 0.7975 - loss: 0.5771 
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7049 - auc: 0.7917 - loss: 0.5812 
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7434 - auc: 0.8092 - loss: 0.5431 
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7926 - auc: 0.8415 - loss: 0.5045 
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7693 - auc: 0.8420 - loss: 0.5160 
Epoch 11/20
16/16 

<bound method TensorFlowTrainer.evaluate of <Sequential name=sequential_34, built=True>>

Выводы. 
Модель обучается, если данные не имеют пропусков.
Если в слое достаточно нейронов, дальнейшее добавление не приносит улучшения метрик.
Минимальное количество слоев - 2. Но можно и 3, результат почти тот же.
От запуска к запуску обучения метрики на выходе разные.
